# Inputs

In [51]:
import tensorflow as tf

import numpy as np
import os
from pathlib import Path

In [53]:
AUTOTUNE = tf.data.experimental.AUTOTUNE

# Data

In [6]:
SCALE_SIZE = 224

In [35]:
DATA_DIR = Path('../input/beeVSwasp/')
BEE_DIR = Path('../input/beeVSwasp/bee/')
WASP_DIR = Path('../input/beeVSwasp/wasp//')

bee_files = sorted(list(BEE_DIR.rglob('*.jpg')))
bee_files = [str(x) for x in bee_files]

wasp_files = sorted(list(WASP_DIR.rglob('*.jpg')))
wasp_files = [str(x) for x in wasp_files]

In [45]:
BEE_SIZE = len(bee_files)
WASP_SIZE = len(wasp_files)
print(len(bee_files), len(wasp_files))

2469 2126


In [41]:
def preprocess_image(image):
    image = tf.image.decode_jpeg(image, channels=3)
    image = tf.image.resize(image, [SCALE_SIZE, SCALE_SIZE])
    image /= 255.0  # normalize to [0,1] range

    return image

def load_and_preprocess_image(path):
    image = tf.io.read_file(path)
    return preprocess_image(image)

1 == Bee

0 == Wasp

In [58]:
labels = [1 for _ in range(BEE_SIZE)] + [0 for _ in range(WASP_SIZE)]
files = bee_files + wasp_files

In [59]:
path_ds = tf.data.Dataset.from_tensor_slices(files)

In [60]:
image_ds = path_ds.map(load_and_preprocess_image, num_parallel_calls=AUTOTUNE)

In [62]:
label_ds = tf.data.Dataset.from_tensor_slices(tf.cast(labels, tf.int64))

In [75]:
image_label_ds = tf.data.Dataset.zip((image_ds, label_ds))

In [77]:
BATCH_SIZE = 32

# Установка размера буфера перемешивания, равного набору данных, гарантирует
# полное перемешивание данных.
ds = image_label_ds.shuffle(buffer_size=BEE_SIZE+WASP_SIZE)
ds = ds.repeat()
ds = ds.batch(BATCH_SIZE)
# `prefetch` позволяет датасету извлекать пакеты в фоновом режиме, во время обучения модели.
ds = ds.prefetch(buffer_size=AUTOTUNE)
ds

<PrefetchDataset shapes: ((None, 224, 224, 3), (None,)), types: (tf.float32, tf.int64)>

In [82]:
ds = image_label_ds.apply(
  tf.data.experimental.shuffle_and_repeat(buffer_size=BEE_SIZE+WASP_SIZE))
ds = ds.batch(BATCH_SIZE)
ds = ds.prefetch(buffer_size=AUTOTUNE)

In [83]:
image_batch, label_batch = next(iter(ds))

# Model